In [64]:
import zipfile36
import os
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words("russian")
import re
from nltk.tokenize import word_tokenize
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
zip_corpus = zipfile36.ZipFile('corpus_news.zip', 'r')
zip_corpus.extractall('n_corpus')

In [45]:
def extract_data(info):
    data = []
    for file in os.listdir(info):
        text = open(os.path.join(info, file), 'r', encoding='utf-8').read()
        data.append(text)
    return data

In [46]:
culture = extract_data('n_corpus/corpus/culture')
hi_tech = extract_data('n_corpus/corpus/hi-tech')
politics = extract_data('n_corpus/corpus/politics')
science = extract_data('n_corpus/corpus/science')

In [47]:
df_culture = pd.DataFrame({'text':culture, 'subcorpus':1})
df_hi_tech = pd.DataFrame({'text':hi_tech, 'subcorpus':2})
df_politics = pd.DataFrame({'text':politics, 'subcorpus':3})
df_science = pd.DataFrame({'text':science, 'subcorpus':4})

In [48]:
data = pd.concat([df_culture, df_hi_tech, df_politics, df_science])

In [49]:
data

,text,subcorpus
0,"""С видом на жизнь"": главные вехи человеческого...",1
1,Ученые признали опасным для здоровья запойный ...,1
2,Еще четверо российских артистов попали в черны...,1
3,Следком: Дмитрию Марьянову кололи сильные преп...,1
4,Александр Галибин объяснился жене в любви чере...,1
...,...,...
206,"Новый антибиотик, сражающий опасные бактерии н...",4
207,"Мисуркин и Шкаплеров вернулись на МКС, побив к...",4
208,На Большом адронном коллайдере обнаружена част...,4
209,"В самой сухой пустыне мира, очень похожей на м...",4


In [65]:
punct = re.compile("[^\w\s]")

def clean_text(text):
    text = re.sub(punct, "", text)
    text = text.lower()
    info_tokenized = word_tokenize(text)
    info_lemmatized = []
    for token in info_tokenized:
        lemma = morph.parse(token)[0]
        lemma = lemma.normal_form
        info_lemmatized.append(lemma)
    info_filtered = []
    for lemma in info_lemmatized:
        if lemma not in stop_words:
            info_filtered.append(lemma)
    cleaned_text = []
    for word in info_filtered:
        cleaned_text.append(word)
    cleaned_text = ' '.join(cleaned_text)
    return cleaned_text

In [66]:
data['text'] = data['text'].apply(clean_text)

In [67]:
data

,text,subcorpus
0,вид жизнь главный веха человеческий путь докум...,1
1,учёный признать опасный здоровье запойный прос...,1
2,ещё четверо российский артист попасть чёрный с...,1
3,следок дмитрий марьянов колоть сильный препара...,1
4,александр галибин объясниться жена любовь стек...,1
...,...,...
206,новый антибиотик сражать опасный бактерия напо...,4
207,мисуркина шкаплер вернуться мкс побить космиче...,4
208,большой адронный коллайдер обнаружить частица ...,4
209,сухой пустыня мир очень похожий марсианский пр...,4


In [ ]:
NAIVE BAYES

In [68]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['subcorpus'], test_size=0.2)

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                     ])

text_clf.fit(X_train, y_train)


predicted = text_clf.predict(X_test)

print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

           1       1.00      0.36      0.53        39
           2       1.00      0.94      0.97        52
           3       0.75      1.00      0.85        82
           4       0.97      0.97      0.97        38

    accuracy                           0.86       211
   macro avg       0.93      0.82      0.83       211
weighted avg       0.90      0.86      0.84       211

